# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [4]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [ ]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

In [ ]:
moving_name = 'AtlasV8'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = fetch_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

In [ ]:
fixed_all

In [ ]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

In [ ]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 18.2658167690059

In [ ]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')


In [ ]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
sc_com = np.array([1.119232944161371051e+04,3.845036998545344431e+03,4.365074759237448234e+03])
scale = np.array([0.452, 0.452, 20])
from_disk = sc_com/scale
print(from_disk)
from_program = np.array([24818.103954839404, 8748.195805110985, 222.0769230769231])
from_disk - from_program

In [7]:
import os
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/Allen'
allenpath = os.path.join(reg_path, 'Allen_10um_sagittal_padded.tif')
allen_arr = read_image(allenpath)
print(allen_arr.dtype, allen_arr.shape)

uint16 (1140, 1000, 1820)


In [11]:
change_z = 10/2
change_y = 10/1.8
change_x = 10/1.8
change = (change_z, change_y, change_x)
print(f'change_z={change_z} change_y={change_y} change_x={change_x}')

change_z=5.0 change_y=5.555555555555555 change_x=5.555555555555555


In [12]:
from scipy.ndimage import zoom
zoomed = zoom(allen_arr, change)
print(zoomed.dtype, zoomed.shape)

MemoryError: Unable to allocate 596. GiB for an array with shape (5700, 5556, 10111) and data type uint16

In [10]:
outpath = os.path.join(reg_path, 'Allen_8x7.2x7.2um_sagittal.tif')
write_image(outpath, zoomed)